IMPORTING NECESSARY PACKAGES

In [67]:
from PIL.Image import ImageTransformHandler
import cv2 as cv
import numpy as np
import pytesseract

SPECIFYING THE PATH WHERE TESSERACT IS INSTALLED

In [68]:
pytesseract.pytesseract.tesseract_cmd = "C:/Program Files (x86)/Tesseract-OCR/tesseract.exe"

DETECTING THE LICENCE PLATES USING HAAR-LIKE FEATURES 

In [69]:
cascade = cv.CascadeClassifier("haarcascade_russian_plate_number.xml")

DEFINING ALL INDIAN STATE CODES FOR VEHICLES

In [70]:
state_codes = {  "AN":"Andaman and Nicobar",
            "AP":"Andhra Pradesh", "AR":"Arunachal Pradesh",
            "AS":"Assam", "BR":"Bihar", "CH":"Chandigarh",
            "DN":"Dadra and Nagar Haveli", "DD":"Daman and Diu",
            "DL":"Delhi", "GA":"Goa", "GJ":"Gujarat",
            "HR":"Haryana", "HP":"Himachal Pradesh",
            "JK":"Jammu and Kashmir", "KA":"Karnataka", "KL":"Kerala",
            "LD":"Lakshadweep", "MP":"Madhya Pradesh", "MH":"Maharashtra", "MN":"Manipur",
            "ML":"Meghalaya", "MZ":"Mizoram", "NL":"Nagaland", "OD":"Odissa",
            "PY":"Pondicherry", "PN":"Punjab", "RJ":"Rajasthan", "SK":"Sikkim", "TN":"TamilNadu",
            "TR":"Tripura", "UP":"Uttar Pradesh", "WB":"West Bengal", "CG":"Chhattisgarh",
            "TS":"Telangana", "JH":"Jharkhand", "UK":"Uttarakhand" }

LICENCE PLATE DETECTION AND PLATE NUMBER RECOGNITION

In [71]:
def Get_NumberPlate(img_filename):
    # READING THE IMAGE
    img = cv.imread(img_filename)

    # CONVERTING TO GRAYSCALE
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # DETECTING THE NUMBER PLATE USING HAAR-LIKE FEATURES
    number_plate = cascade.detectMultiScale(gray)
    
    # CROPPING THE NUMBER PLATE ALONE FROM THE GRAYSCALE IMAGE
    for (x,y,w,h) in number_plate:  # SPLITTING UP THE BOUNDING BOXES INTO (x,y,w,h) VARIABLES
        wT,hT,cT = img.shape  # wT -> WIDTH | hT -> HEIGHT | cT -> CHANNEL OF THE IMAGE
        a,b = (int(0.02*wT), int(0.02*hT))
        plate = img[y+a : y+h-a, x+b : x+w-b, :]

        # IMAGE PROCESSING
        # MAKING THE NUMBER PLATE LOOK MORE DARKER
        kernel = np.ones((1,1), np.uint8)  # CONVOLVING THE IMAGE WITH AN ODD SIZE MATRIX 

        # DILATION INCREASES THE WHITE REGION OF THE NUMBER PLATE
        plate = cv.dilate(plate, kernel, iterations = 1)

        # ERODE FUNCTION REMOVES SMALL WHITE NOISES AND DETACHES TWO CONNECTED OBJECTS 
        plate = cv.erode(plate, kernel, iterations = 1)
        
        # CONVERTING THE NUMBER PLATE TO GRAYSCALE 
        plate_gray = cv.cvtColor(plate, cv.COLOR_BGR2GRAY)

        # APPLYING THRESHOLDING TECHNIQUE ON THE GRAYSCALE IMAGE FOR BETTER SEPARATION OF FOREGROUND OBJECT FROM BACKGROUND PIXELS
        (thresh, plate) = cv.threshold(plate_gray, 127, 255, cv.THRESH_BINARY)

        # EXTRACTING TEXT FROM THE NUMBER PLATE USING 'pytesseract'(AN OCR TOOL)
        text = pytesseract.image_to_string(plate)

        # REMOVING BLANK SPACES FROM THE TEXT
        text = ''.join(e for e in text if e.isalnum())

        # GETTING THE STATE CODE OF THE VEHICLE
        state = text[0:2]

        # FETCHING STATE DETAILS BASED ON THE STATE CODE OF THE VEHICLE
        try:
            print('CAR BELONGS TO: ', state_codes[state])
        except:
            print('STATE NOT RECOGNISED')

        print("CAR NUMBER: ", text)
        
        # DETECTING THE NUMBER PLATE AREA ON THE IMAGE
        cv.rectangle(img, (x,y), (x+w,y+h), (51,51,255), 2)
        cv.rectangle(img, (x-1,y-40), (x+w+1,y), (51,51,255), -1)

        # DISPLAYING THE OCR OUTPUT ON THE IMAGE ITSELF
        cv.putText(img, text, (x,y-10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)

        # DISPLAYING THE NUMBER PLATE
        cv.imshow("plate", plate)

    # STORING THE OUTPUT IMAGE CONTAINING OCR TEXT OF THE NUMBER PLATE    
    cv.imwrite("Output_Image.png", img)
    cv.imwrite("NumberPlate.png", plate)

    # DISPLAYING THE OUTPUT IMAGE
    cv.imshow("Result", img)

    # EXITING THE OPERATION WHEN 'q' IS PRESSED
    if cv.waitKey(1) & 0xFF == ord('q'):
        exit()
    
    # CLEANING UP ALL OPENED WINDOWS
    cv.destroyAllWindows()

CALLING THE FUNCTION BY PASSING AN IMAGE OF A CAR AS INPUT

In [72]:
Get_NumberPlate("Car_Image.png")

CAR BELONGS TO:  Kerala
CAR NUMBER:  KL5tk4999
